Madelaine Zinser

Lyn Nguyen 

December 2022

In [73]:
import pandas as pd
import seaborn as sns

sns.set(style="whitegrid")

## Table of Content <a class="anchor" id="content"></a>
* [ETL](#etl)
* [EDA](#eda)
    * [Single Variable EDA](#single-var-eda)
        - [popularity](#popularity): 
        - [acousticness](#acousticness)
        - [danceability](#dance)
        - [duration_ms](#duration)
        - [energy](#energy)
        - [instrumentalness](#instr)
        - [liveness](#live)
        - [loudness](#loud)
        - [speechiness](#speech)
        - [tempo](#tempo)
        - [valance](#valance)
     
    * [Pairwise EDA](#pairwise-eda)
* [Build Model](#build-model)
* [Validate Model](#validate-model)
* [Use Model](#predict)

## ELT <a class="anchor" id="etl"></a>

We are working with 2 tables `tweets` and `users`. `scrapers.py` outputs the data. 

We will need to combine and remove duplicates.
- tweets1.csv and users1.csv - Nov 30 - Dec 1
- tweets2.csv and users2.csv - Nov 30 - Dec 2
- tweets3.csv and users3.csv - Nov 29 - Dec 5 pulled on Dec 6.

In [ ]:
tweets = pd.read_csv("data/tweets2.csv")
users = pd.read_csv("data/users2.csv")

In [75]:
print(len(tweets))
tweets[0:2]

454


,experiment_id,experiment_group,text,tweet_id,likes,retweets,created_at,user_id,in_reply_to_status_id,in_reply_to_user_id,in_reply_to_screen_name,screen_name
0,0,usedgov,@usedgov Thank you for loan forgiveness,1599146922790354944,1,0,Sat Dec 03 21:01:51 +0000 2022,1508197172080222214,1.597260e+18,20437286,usedgov,luzbaez628
1,1,usedgov,@usedgov @SecCardona @usesgov @potus #studentl...,1598411747349913601,0,0,Thu Dec 01 20:20:32 +0000 2022,1477370850147983367,1.595158e+18,20437286,usedgov,fabulosi_t


In [82]:
print(len(users))
users[0:2]

454


,user_id,created_at,description,location,followers_count,screen_name,statuses_count,favourites_count,verified
0,1508197172080222214,Sun Mar 27 21:41:16 +0000 2022,Thoughtful,NaN,0,luzbaez628,31,3,False
1,1477370850147983367,Sat Jan 01 20:07:25 +0000 2022,Everything I do is done with fabulosiT.,NaN,74,fabulosi_t,5111,5524,False


There are 454 datapoints split between the 3 sources (Dept of Education, CNN, Fox News). Currently our database is constraint to only data between Nov.27 and Dec.4.

The columns of `tweets` are:
- experiment_id
- experiment_group 
- text
- tweet_id
- likes
- retweets
- created_at
- user_id
- in_reply_to_status_id
- in_reply_to_user_id
- in_reply_to_screen_name
- screen_name: of the author

`users` have the following columns: 
- user_id 
- created_at 
- description 
- location 
- followers_count 
- screen_name 
- statuses_count 
- favourites_count 
- verified


## EDA <a class="anchor" id="eda"></a>

###  Single Variable EDA <a class="anchor" id="single-var-eda"></a>

#### in_reply_to_screen_name <a class="anchor" id="experiment_group"></a>

Note that 'experiment_group' and 'in_reply_to_screen_name' identify the same value.

What is the share in non-retweet replies by experiment groups? 

In [93]:
tfreq = pd.DataFrame(
    tweets
        .groupby('in_reply_to_screen_name')
        .size()
        .reset_index(name="count")
)
tfreq

,in_reply_to_screen_name,count
0,CNN,17
1,FoxNews,430
2,usedgov,7


Out of 454 tweets, 430 are engagement on FoxNews. Surprisingly, US Department of Education have only 7 replies. How many unique authors are there? 

In [95]:
ufreq = pd.DataFrame(
    tweets
        .groupby('screen_name')
        .size()
        .reset_index(name="count")
        .sort_values("count")
)
len(ufreq)
#user with more screen names
ufreq[ufreq['count'] >1]

,screen_name,count
321,fabulosi_t,2
192,PCopposition,2
266,Vincent08571545,2
63,DahlmanCarl,2
90,Forevertrue10,2
54,Curious56388405,2
68,DavidWo15416040,3
418,thomaslew13,3
168,Marcos_Swagga,4
282,argo_times,6


- What are the profiles of these 10 authors? 

- Does any of them engage with more than one experiment group? 

- Is their engagement due to high "popularity"? 

- Where do their tweets stand in the "average" tweet favorite count and retweet counts? 

### Users

In [78]:
users[0:2]

,user_id,created_at,description,location,followers_count,screen_name,statuses_count,favourites_count,verified
0,1508197172080222214,Sun Mar 27 21:41:16 +0000 2022,Thoughtful,NaN,0,luzbaez628,31,3,False
1,1477370850147983367,Sat Jan 01 20:07:25 +0000 2022,Everything I do is done with fabulosiT.,NaN,74,fabulosi_t,5111,5524,False


In [79]:
users.columns

Index(['user_id', 'created_at', 'description', 'location', 'followers_count',
       'screen_name', 'statuses_count', 'favourites_count', 'verified'],
      dtype='object')